In [11]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.1'
#spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists... Done


In [12]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-11-04 15:59:57--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.04MB/s    in 0.8s    

2020-11-04 15:59:59 (1.04 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [14]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [15]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
#url="https://<bucket name>.s3.amazonaws.com/user_data.csv"
url="https://ejbigdatasets.s3.amazonaws.com/UrbanSound8K/metadata/UrbanSound8K.csv"
spark.sparkContext.addFile(url)
sound_df = spark.read.csv(SparkFiles.get("UrbanSound8K.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
# sound_df.show()

# Print schema
sound_df.printSchema()

root
 |-- slice_file_name: string (nullable = true)
 |-- fsID: integer (nullable = true)
 |-- start: double (nullable = true)
 |-- end: double (nullable = true)
 |-- salience: integer (nullable = true)
 |-- fold: integer (nullable = true)
 |-- classID: integer (nullable = true)
 |-- class: string (nullable = true)



In [16]:
classnames_df = sound_df.select('classID', 'class').distinct().orderBy(sound_df.classID)
classnames_df.show()

+-------+----------------+
|classID|           class|
+-------+----------------+
|      0| air_conditioner|
|      1|        car_horn|
|      2|children_playing|
|      3|        dog_bark|
|      4|        drilling|
|      5|   engine_idling|
|      6|        gun_shot|
|      7|      jackhammer|
|      8|           siren|
|      9|    street_music|
+-------+----------------+



In [17]:
from pyspark.sql.functions import col
from pyspark.sql.functions import monotonically_increasing_id 

In [18]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://urbansounddb.cwkue3lwi5mx.us-east-1.rds.amazonaws.com:5432/postgres"
config = {"user":"root", 
          "password": "urbansoundDB", 
          "driver":"org.postgresql.Driver"}


In [ ]:
# Write dataframe to class_names table in RDS
classnames_df.write.jdbc(url=jdbc_url, table='class_names', mode=mode, properties=config)

In [20]:
# Write dataframe to fold{1~10}_metadata table in RDS

num_folds = 10
for i in range(1,num_folds+1):
  fold_df = sound_df.filter(sound_df['fold']==i).select('slice_file_name', 'salience', 'classID')
  fold_df = fold_df.select("*").withColumn("id", monotonically_increasing_id())
  fold_df = fold_df.select('id', 'slice_file_name', 'salience', 'classID')

  # Write DataFrame to fold{i}_metadata table in RDS
  table_name = f"fold{i}_metadata"
  fold_df.write.jdbc(url=jdbc_url, table=table_name, mode=mode, properties=config)

In [21]:
sound_df.show()

+------------------+------+---------+---------+--------+----+-------+----------------+
|   slice_file_name|  fsID|    start|      end|salience|fold|classID|           class|
+------------------+------+---------+---------+--------+----+-------+----------------+
|  100032-3-0-0.wav|100032|      0.0| 0.317551|       1|   5|      3|        dog_bark|
|100263-2-0-117.wav|100263|     58.5|     62.5|       1|   5|      2|children_playing|
|100263-2-0-121.wav|100263|     60.5|     64.5|       1|   5|      2|children_playing|
|100263-2-0-126.wav|100263|     63.0|     67.0|       1|   5|      2|children_playing|
|100263-2-0-137.wav|100263|     68.5|     72.5|       1|   5|      2|children_playing|
|100263-2-0-143.wav|100263|     71.5|     75.5|       1|   5|      2|children_playing|
|100263-2-0-161.wav|100263|     80.5|     84.5|       1|   5|      2|children_playing|
|  100263-2-0-3.wav|100263|      1.5|      5.5|       1|   5|      2|children_playing|
| 100263-2-0-36.wav|100263|     18.0|     2